# **Modified VGG16 with Inception Block for Cancer Classification**
This Colab notebook implements a modified **VGG16** model for binary classification (benign vs. malignant) using the **BreaKHis** dataset. The model includes:
- **Pre-trained VGG16** (with frozen convolutional layers).
- **Custom Inception Block** added after VGG16 for feature extraction.
- **Dense Layer for classification**.

### **Dataset Details**
- **BreaKHis dataset** (40X magnification).
- Images are loaded from Google Drive.
- Labels: **Benign (0)** and **Malignant (1)**.

### **Data Split**
- **60% Training**
- **10% Validation**
- **30% Testing**

### **Training Details**
- Image preprocessing: **224x224 resizing & normalization**.
- **Adam Optimizer** with Binary Cross-Entropy Loss.
- **Batch Size: 16**, **Epochs: 10**.

### **Evaluation Metrics**
- **Accuracy**
- **F1 Score**
- **Geometric Mean Score (G-Mean)**
- **Informedness (IBA)**

***
**Magnification Factor: 40X**
***

In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, concatenate, AveragePooling2D, Input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix
from imblearn.metrics import geometric_mean_score

# Define dataset paths
benign_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/benign/40X"
malignant_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/malignant/40X"

def load_image_paths(dir_path):
    return [os.path.join(dir_path, img) for img in os.listdir(dir_path) if img.endswith('.png')]

benign_images = load_image_paths(benign_dir)
malignant_images = load_image_paths(malignant_dir)

print(f"Total Benign Images: {len(benign_images)}")
print(f"Total Malignant Images: {len(malignant_images)}")

benign_labels = [0] * len(benign_images)
malignant_labels = [1] * len(malignant_images)

all_images = np.array(benign_images + malignant_images)
all_labels = np.array(benign_labels + malignant_labels)

# Split dataset (60% train, 10% val, 30% test)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.3, stratify=all_labels, random_state=42)
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.1429, stratify=train_labels, random_state=42)

print(f"Training samples: {len(train_images)}")
print(f"Validation samples: {len(val_images)}")
print(f"Testing samples: {len(test_images)}")

def process_path(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    img = img / 255.0
    return img, label

BATCH_SIZE = 128

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).map(process_path).shuffle(1000).batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels)).map(process_path).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).map(process_path).batch(BATCH_SIZE)

if sum(1 for _ in test_dataset) == 0:
    raise ValueError("Testing dataset is empty. Adjust your dataset split.")

# Load VGG16 without top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

def inception_block(x):
    branch1 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)

    branch2 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)
    branch2 = Conv2D(128, (3, 3), activation='relu', padding='same')(branch2)

    branch3 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)
    branch3 = Conv2D(128, (5, 5), activation='relu', padding='same')(branch3)

    branch4 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch4 = Conv2D(64, (1, 1), activation='relu', padding='same')(branch4)

    output = concatenate([branch1, branch2, branch3, branch4], axis=-1)
    return output

# Add Inception block after VGG16
x = inception_block(base_model.output)
x = AveragePooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=x)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

EPOCHS = 100
history = model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS)

test_preds = model.predict(test_dataset)
test_preds = (test_preds > 0.5).astype(int).flatten()

tn, fp, fn, tp = confusion_matrix(test_labels, test_preds).ravel()
iba = (tp / (tp + fn)) + (tn / (tn + fp)) - 1

f1 = f1_score(test_labels, test_preds)
gmean = geometric_mean_score(test_labels, test_preds)

loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"G-Mean: {gmean:.4f}")
print(f"Informedness (IBA): {iba:.4f}")

Total Benign Images: 626
Total Malignant Images: 1370
Training samples: 1197
Validation samples: 200
Testing samples: 599
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 242s 8s/step - accuracy: 0.5590 - loss: 0.9462 - val_accuracy: 0.6850 - val_loss: 0.6423
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 42s 829ms/step - accuracy: 0.6869 - loss: 0.6414 - val_accuracy: 0.7000 - val_loss: 0.6006
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.7023 - loss: 0.5894 - val_accuracy: 0.6900 - val_loss: 0.5523
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 21s 771ms/step - accuracy: 0.7264 - loss: 0.5540 - val_accuracy: 0.7650 - val_loss: 0.5056
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 827ms/step - accuracy: 0.7763 - loss: 0.5081 - val_accuracy: 0.8350 - val_loss: 0.4550
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 19s 805ms/step - accuracy: 0.7715 - loss: 0.4865 - val_accuracy: 0.8000 - val_loss: 0.4572
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step -

***
**Magnification Factor: 100X**
***

In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, concatenate, AveragePooling2D, Input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix
from imblearn.metrics import geometric_mean_score

# Define dataset paths
benign_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/benign/100X"
malignant_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/malignant/100X"

def load_image_paths(dir_path):
    return [os.path.join(dir_path, img) for img in os.listdir(dir_path) if img.endswith('.png')]

benign_images = load_image_paths(benign_dir)
malignant_images = load_image_paths(malignant_dir)

print(f"Total Benign Images: {len(benign_images)}")
print(f"Total Malignant Images: {len(malignant_images)}")

benign_labels = [0] * len(benign_images)
malignant_labels = [1] * len(malignant_images)

all_images = np.array(benign_images + malignant_images)
all_labels = np.array(benign_labels + malignant_labels)

# Split dataset (60% train, 10% val, 30% test)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.3, stratify=all_labels, random_state=42)
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.1429, stratify=train_labels, random_state=42)

print(f"Training samples: {len(train_images)}")
print(f"Validation samples: {len(val_images)}")
print(f"Testing samples: {len(test_images)}")

def process_path(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    img = img / 255.0
    return img, label

BATCH_SIZE = 128

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).map(process_path).shuffle(1000).batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels)).map(process_path).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).map(process_path).batch(BATCH_SIZE)

if sum(1 for _ in test_dataset) == 0:
    raise ValueError("Testing dataset is empty. Adjust your dataset split.")

# Load VGG16 without top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

def inception_block(x):
    branch1 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)

    branch2 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)
    branch2 = Conv2D(128, (3, 3), activation='relu', padding='same')(branch2)

    branch3 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)
    branch3 = Conv2D(128, (5, 5), activation='relu', padding='same')(branch3)

    branch4 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch4 = Conv2D(64, (1, 1), activation='relu', padding='same')(branch4)

    output = concatenate([branch1, branch2, branch3, branch4], axis=-1)
    return output

# Add Inception block after VGG16
x = inception_block(base_model.output)
x = AveragePooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=x)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

EPOCHS = 100
history = model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS)

test_preds = model.predict(test_dataset)
test_preds = (test_preds > 0.5).astype(int).flatten()

tn, fp, fn, tp = confusion_matrix(test_labels, test_preds).ravel()
iba = (tp / (tp + fn)) + (tn / (tn + fp)) - 1

f1 = f1_score(test_labels, test_preds)
gmean = geometric_mean_score(test_labels, test_preds)

loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"G-Mean: {gmean:.4f}")
print(f"Informedness (IBA): {iba:.4f}")

Total Benign Images: 649
Total Malignant Images: 1437
Training samples: 1251
Validation samples: 209
Testing samples: 626
Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 243s 11s/step - accuracy: 0.5227 - loss: 1.0397 - val_accuracy: 0.6890 - val_loss: 0.6168
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.6880 - loss: 0.6283 - val_accuracy: 0.6890 - val_loss: 0.6092
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 23s 932ms/step - accuracy: 0.7005 - loss: 0.5917 - val_accuracy: 0.6890 - val_loss: 0.5674
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 908ms/step - accuracy: 0.7043 - loss: 0.5526 - val_accuracy: 0.6890 - val_loss: 0.5262
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 21s 942ms/step - accuracy: 0.7524 - loss: 0.4998 - val_accuracy: 0.8182 - val_loss: 0.4431
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 923ms/step - accuracy: 0.8206 - loss: 0.4255 - val_accuracy: 0.8182 - val_loss: 0.3861
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 924ms/step - accuracy: 0.8373 - loss: 0.3812 - val_accuracy

***
**Magnification Factor: 200X**
***

In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, concatenate, AveragePooling2D, Input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix
from imblearn.metrics import geometric_mean_score

# Define dataset paths
benign_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/benign/200X"
malignant_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/malignant/200X"

def load_image_paths(dir_path):
    return [os.path.join(dir_path, img) for img in os.listdir(dir_path) if img.endswith('.png')]

benign_images = load_image_paths(benign_dir)
malignant_images = load_image_paths(malignant_dir)

print(f"Total Benign Images: {len(benign_images)}")
print(f"Total Malignant Images: {len(malignant_images)}")

benign_labels = [0] * len(benign_images)
malignant_labels = [1] * len(malignant_images)

all_images = np.array(benign_images + malignant_images)
all_labels = np.array(benign_labels + malignant_labels)

# Split dataset (60% train, 10% val, 30% test)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.3, stratify=all_labels, random_state=42)
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.1429, stratify=train_labels, random_state=42)

print(f"Training samples: {len(train_images)}")
print(f"Validation samples: {len(val_images)}")
print(f"Testing samples: {len(test_images)}")

def process_path(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    img = img / 255.0
    return img, label

BATCH_SIZE = 128

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).map(process_path).shuffle(1000).batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels)).map(process_path).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).map(process_path).batch(BATCH_SIZE)

if sum(1 for _ in test_dataset) == 0:
    raise ValueError("Testing dataset is empty. Adjust your dataset split.")

# Load VGG16 without top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

def inception_block(x):
    branch1 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)

    branch2 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)
    branch2 = Conv2D(128, (3, 3), activation='relu', padding='same')(branch2)

    branch3 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)
    branch3 = Conv2D(128, (5, 5), activation='relu', padding='same')(branch3)

    branch4 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch4 = Conv2D(64, (1, 1), activation='relu', padding='same')(branch4)

    output = concatenate([branch1, branch2, branch3, branch4], axis=-1)
    return output

# Add Inception block after VGG16
x = inception_block(base_model.output)
x = AveragePooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=x)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

EPOCHS = 100
history = model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS)

test_preds = model.predict(test_dataset)
test_preds = (test_preds > 0.5).astype(int).flatten()

tn, fp, fn, tp = confusion_matrix(test_labels, test_preds).ravel()
iba = (tp / (tp + fn)) + (tn / (tn + fp)) - 1

f1 = f1_score(test_labels, test_preds)
gmean = geometric_mean_score(test_labels, test_preds)

loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"G-Mean: {gmean:.4f}")
print(f"Informedness (IBA): {iba:.4f}")

Total Benign Images: 623
Total Malignant Images: 1390
Training samples: 1207
Validation samples: 202
Testing samples: 604
Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 295s 13s/step - accuracy: 0.6067 - loss: 0.8259 - val_accuracy: 0.6881 - val_loss: 0.6153
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 19s 775ms/step - accuracy: 0.6914 - loss: 0.6017 - val_accuracy: 0.6881 - val_loss: 0.5767
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 18s 788ms/step - accuracy: 0.6948 - loss: 0.5619 - val_accuracy: 0.7228 - val_loss: 0.5243
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 21s 788ms/step - accuracy: 0.7715 - loss: 0.4860 - val_accuracy: 0.8069 - val_loss: 0.4833
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 19s 817ms/step - accuracy: 0.8205 - loss: 0.4328 - val_accuracy: 0.7673 - val_loss: 0.4318
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.8412 - loss: 0.3654 - val_accuracy: 0.7772 - val_loss: 0.4238
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 18s 769ms/step - accuracy: 0.8466 - loss: 0.3319 - val_accuracy

4/5 ━━━━━━━━━━━━━━━━━━━━ 1s 2s/step

5/5 ━━━━━━━━━━━━━━━━━━━━ 36s 8s/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.8696 - loss: 0.9011
Test Accuracy: 0.8742
F1 Score: 0.9100
G-Mean: 0.8421
Informedness (IBA): 0.6909


***
**Magnification Factor: 400X**
***

In [3]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, concatenate, AveragePooling2D, Input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix
from imblearn.metrics import geometric_mean_score

# Define dataset paths
benign_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/benign/400X"
malignant_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/malignant/400X"

def load_image_paths(dir_path):
    return [os.path.join(dir_path, img) for img in os.listdir(dir_path) if img.endswith('.png')]

benign_images = load_image_paths(benign_dir)
malignant_images = load_image_paths(malignant_dir)

print(f"Total Benign Images: {len(benign_images)}")
print(f"Total Malignant Images: {len(malignant_images)}")

benign_labels = [0] * len(benign_images)
malignant_labels = [1] * len(malignant_images)

all_images = np.array(benign_images + malignant_images)
all_labels = np.array(benign_labels + malignant_labels)

# Split dataset (60% train, 10% val, 30% test)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.3, stratify=all_labels, random_state=42)
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.1429, stratify=train_labels, random_state=42)

print(f"Training samples: {len(train_images)}")
print(f"Validation samples: {len(val_images)}")
print(f"Testing samples: {len(test_images)}")

def process_path(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    img = img / 255.0
    return img, label

BATCH_SIZE = 128

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).map(process_path).shuffle(1000).batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels)).map(process_path).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).map(process_path).batch(BATCH_SIZE)

if sum(1 for _ in test_dataset) == 0:
    raise ValueError("Testing dataset is empty. Adjust your dataset split.")

# Load VGG16 without top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

def inception_block(x):
    branch1 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)

    branch2 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)
    branch2 = Conv2D(128, (3, 3), activation='relu', padding='same')(branch2)

    branch3 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)
    branch3 = Conv2D(128, (5, 5), activation='relu', padding='same')(branch3)

    branch4 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch4 = Conv2D(64, (1, 1), activation='relu', padding='same')(branch4)

    output = concatenate([branch1, branch2, branch3, branch4], axis=-1)
    return output

# Add Inception block after VGG16
x = inception_block(base_model.output)
x = AveragePooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=x)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

EPOCHS = 100
history = model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS)

test_preds = model.predict(test_dataset)
test_preds = (test_preds > 0.5).astype(int).flatten()

tn, fp, fn, tp = confusion_matrix(test_labels, test_preds).ravel()
iba = (tp / (tp + fn)) + (tn / (tn + fp)) - 1

f1 = f1_score(test_labels, test_preds)
gmean = geometric_mean_score(test_labels, test_preds)

loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"G-Mean: {gmean:.4f}")
print(f"Informedness (IBA): {iba:.4f}")

Total Benign Images: 588
Total Malignant Images: 1232
Training samples: 1091
Validation samples: 183
Testing samples: 546
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 169s 7s/step - accuracy: 0.5317 - loss: 0.9610 - val_accuracy: 0.6776 - val_loss: 0.6309
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 57s 1s/step - accuracy: 0.6745 - loss: 0.6338 - val_accuracy: 0.6776 - val_loss: 0.6150
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 17s 795ms/step - accuracy: 0.6691 - loss: 0.6111 - val_accuracy: 0.6776 - val_loss: 0.5851
Epoch 4/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 19s 826ms/step - accuracy: 0.6855 - loss: 0.5692 - val_accuracy: 0.7541 - val_loss: 0.5362
Epoch 5/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 20s 901ms/step - accuracy: 0.7435 - loss: 0.5358 - val_accuracy: 0.7978 - val_loss: 0.4833
Epoch 6/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - accuracy: 0.7929 - loss: 0.4547 - val_accuracy: 0.8033 - val_loss: 0.4348
Epoch 7/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 39s 880ms/step - accuracy: 0.8

# Here is a table summarizing the performance metrics for all magnification factors (40X, 100X, 200X, 400X) from notebook:

In [4]:
import pandas as pd

# Create a DataFrame with the updated metrics
data = {
    'Magnification': ['40X', '100X', '200X', '400X'],
    'Test Accuracy': [0.8982, 0.8658, 0.8742, 0.8370],
    'F1 Score': [0.9264, 0.9034, 0.9100, 0.8802],
    'G-Mean': [0.8748, 0.8347, 0.8421, 0.8080],
    'Informedness (IBA)': [0.7535, 0.6759, 0.6909, 0.6224]
}

df = pd.DataFrame(data)

# Display the table with formatting
styled_df = df.style \
    .format({
        'Test Accuracy': '{:.4f}',
        'F1 Score': '{:.4f}',
        'G-Mean': '{:.4f}',
        'Informedness (IBA)': '{:.4f}'
    }) \
    .set_properties(**{'text-align': 'center'}) \
    .set_table_styles([{
        'selector': 'th',
        'props': [('background-color', '#000000'), ('font-weight', 'bold')]
    }]) \
    .hide(axis='index')

styled_df


Magnification,Test Accuracy,F1 Score,G-Mean,Informedness (IBA)
40X,0.8982,0.9264,0.8748,0.7535
100X,0.8658,0.9034,0.8347,0.6759
200X,0.8742,0.9100,0.8421,0.6909
400X,0.8370,0.8802,0.8080,0.6224
